Yapılacaklar:
modelin aşırı öğrenmesine izin verip başarı oranını ne kadar yükselebileceğine bakmak.

### Importlar

In [37]:
import pandas as pd
from collections import Counter

import os

import shutil
import torch
import torchvision

import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

from torchvision.models import densenet161, DenseNet161_Weights

from torchvision.models import efficientnet_b2, EfficientNet_B2_Weights

from sklearn.metrics import classification_report

import copy

# import intel_extension_for_pytorch as ipex


# from sklearn.metrics import classification_report
# import matplotlib.pyplot as plt

### Cihaz tercihi

In [38]:
# input_choice = 1
# input_choice = input("Seçim yapın:\n1. CPU için\n2. GPU (CUDA) için\n3. XPU (Intel ARC) için\n (Enter'a basmak CPU seçimi yapar)\n(1/2/3): ")
# if input_choice == '1':
#     device = torch.device("cpu")
# elif input_choice == '2':
#     device = torch.device("cuda")

# elif input_choice == '3':
#     device = torch.device("xpu") 


device = torch.device("cuda")
# device = torch.device("cuda" if torch.xpu.is_available() else "cpu")

In [39]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])  # grayscale için
    ]),
    'validation': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
    'test': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
}

In [43]:
data_dir = 'data_model/'

image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'validation', 'test']
}

dataloaders = {
    # x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2)

    # 4070 için
    x: DataLoader(image_datasets[x], batch_size=64, shuffle=True, num_workers=4)
    for x in ['train', 'validation', 'test']
}

class_names = image_datasets['train'].classes
print("Sınıflar:", class_names)

Sınıflar: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Effusion', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumothorax']


# Fonksiyonlar

### Model eğitecek fonksiyon

In [44]:
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=50, patience=5, class_names=None):
    """
    Derin öğrenme modelini eğitim ve doğrulama verisi üzerinde eğitir.

    Args:
        model (nn.Module): Eğitilecek PyTorch modeli.
        dataloaders (dict): 'train' ve 'validation' anahtarlarına sahip DataLoader sözlüğü.
        criterion (loss): Kayıp fonksiyonu.
        optimizer (torch.optim): Optimizasyon algoritması.
        device (torch.device): Eğitimde kullanılacak cihaz.
        num_epochs (int, optional): Maksimum eğitim dönemi sayısı. Varsayılan 50.
        patience (int, optional): Early stopping için tekrar süresi (doğruluk gelişmezse durma). Varsayılan 5.
        class_names (list, optional): Sınıf isimlerinin listesi. Sınıf bazlı doğruluk ve rapor için kullanılır. Varsayılan None.

    Returns:
        model (nn.Module): En iyi doğrulama başarımına sahip ağırlıklarla geri dönen model.
    """
    best_model_wts = model.state_dict()
    best_acc = 0.0
    epochs_no_improve = 0  # Early stopping için sayaç

    if class_names:
        print("Sınıflar:", class_names)
        num_classes = len(class_names)


    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 20)


        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()   # Eğitim moduna geç
            else:
                model.eval()    # Değerlendirme moduna geç

            running_loss = 0.0
            running_corrects = 0
            all_labels = []
            all_preds = []


            # Her batch için döngü
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                if phase == 'validation':
                    all_labels.extend(labels.cpu().numpy())
                    all_preds.extend(preds.cpu().numpy())


            # Epoch sonu istatistikleri
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # En iyi doğrulama modelini kaydet
            if phase == 'validation':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    epochs_no_improve = 0  # İyileşme varsa sıfırla
                else:
                    epochs_no_improve += 1  # İyileşme yoksa artır
                print(f'{epochs_no_improve} dönem boyunca iyileşme yok.')


                # Sınıf bazlı başarı hesaplama
                if class_names:
                    class_correct = list(0. for i in range(num_classes))
                    class_total = list(0. for i in range(num_classes))

                    for i in range(len(all_labels)):
                        label = all_labels[i]
                        pred = all_preds[i]
                        if label == pred:
                            class_correct[label] += 1
                        class_total[label] += 1


                    print('\nSınıf Bazında Doğruluk:')
                    for i in range(num_classes):
                        if class_total[i] > 0:
                            accuracy = 100 * class_correct[i] / class_total[i]
                            print(f'Sınıf {class_names[i]}: {accuracy:.2f}% ({int(class_correct[i])}/{int(class_total[i])})')
                        else:
                            print(f'Sınıf {class_names[i]}: Hiç örnek bulunamadı.')


                    # Sınıf bazında rapor
                    print('\nSınıf Bazında Rapor:')
                    print(classification_report(all_labels, all_preds))

                else:
                    pass

        # Early stopping kontrolü
        if epochs_no_improve >= patience:
            print(f"\nErken durdurma tetiklendi. Doğrulama doğruluğu {patience} dönemi boyunca gelişmedi.")
            break

    # En iyi ağırlıkları geri yükle
    model.load_state_dict(best_model_wts)
    return model

### Modeli kaydet

In [45]:
def save_model(model, model_name):
    model_save_path = f"{model_name}.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model başarıyla kaydedildi: {model_save_path}")

# SimpleCNN modeli

In [25]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # -> (32, 224, 224)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (32, 112, 112)

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # -> (64, 112, 112)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (64, 56, 56)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * 56 * 56, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [26]:
SimpleCNN_model = SimpleCNN(num_classes=len(class_names)).to(device)

In [27]:
SimpleCNN_criterion = nn.CrossEntropyLoss()
SimpleCNN_optimizer = torch.optim.Adam(SimpleCNN_model.parameters(), lr=0.001)

# SimplerCNN2 Model

In [ ]:
import torch.nn as nn

class SimplerCNN2(nn.Module):
    def __init__(self, num_classes):
        super(SimplerCNN2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # -> (16, 224, 224)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (16, 112, 112)

            nn.Conv2d(16, 32, kernel_size=3, padding=1), # -> (32, 112, 112)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (32, 56, 56)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(32 * 56 * 56, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [ ]:
SimplerCNN2_model = SimplerCNN2(num_classes=len(class_names)).to(device)

In [ ]:
SimplerCNN2_criterion = nn.CrossEntropyLoss()
SimplerCNN2_optimizer = torch.optim.Adam(SimplerCNN2_model.parameters(), lr=0.001)

# CustomCNN Model

In [28]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # -> (32, 224, 224)
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (32, 112, 112)

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (64, 56, 56)

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (128, 28, 28)

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (256, 14, 14)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 14 * 14, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


### Modeli oluşturma

In [29]:
scratch_model = CustomCNN(num_classes=len(class_names)).to(device)

### Loss ve optimizer

In [30]:
scratch_criterion = nn.CrossEntropyLoss()
scratch_optimizer = torch.optim.Adam(scratch_model.parameters(), lr=0.001)

# Resnet18

In [11]:
resnet18_model = models.resnet18(pretrained=True)

# İlk katman RGB (3 kanal) bekliyor. 1 kanala uyarladık.
resnet18_model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Son katmanı sınıf sayısına göre değiştirdik.
num_ftrs = resnet18_model.fc.in_features
resnet18_model.fc = nn.Linear(num_ftrs, len(class_names))

resnet18_model = resnet18_model.to(device)

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
resnet18_criterion = nn.CrossEntropyLoss().to(device)
resnet18_optimizer = optim.Adam(resnet18_model.parameters(), lr=0.001)

# DenseNet161

In [46]:
# Ağırlıkları al
densenet161_weights = DenseNet161_Weights.DEFAULT
densenet161_model = densenet161(weights=densenet161_weights)

# Giriş katmanını 1 kanala uyarlıyoruz
densenet161_model.features.conv0 = nn.Conv2d(1, 96, kernel_size=7, stride=2, padding=3, bias=False)

# Çıkış katmanını sınıf sayısına göre değiştiriyoruz
densenet161_num_ftrs = densenet161_model.classifier.in_features
densenet161_model.classifier = nn.Linear(densenet161_num_ftrs, len(class_names))

# Cihaza gönder
densenet161_model = densenet161_model.to(device)

# Loss ve optimizer
densenet161_criterion = nn.CrossEntropyLoss().to(device)
densenet161_optimizer = optim.Adam(densenet161_model.parameters(), lr=0.001)

# EfficientNet-B2

In [14]:
# En güncel ağırlıkları yükle
efficientnet_b2_weights = EfficientNet_B2_Weights.DEFAULT
efficientnet_b2_model = efficientnet_b2(weights=efficientnet_b2_weights)

# Grayscale (1 kanal) girişe uygun hale getir (ilk katman 3 kanallı bekliyor)
# efficientnet.features[0][0] => ilk Conv2d katmanı
efficientnet_b2_model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)

# Son katmanı sınıf sayısına göre ayarla
efficientnet_b2_num_ftrs = efficientnet_b2_model.classifier[1].in_features
efficientnet_b2_model.classifier[1] = nn.Linear(efficientnet_b2_num_ftrs, len(class_names))

# Cihaza gönder
efficientnet_b2_model = efficientnet_b2_model.to(device)

# Loss ve optimizer
efficientnet_b2_criterion = nn.CrossEntropyLoss()
efficientnet_b2_optimizer = optim.Adam(efficientnet_b2_model.parameters(), lr=0.001)

In [15]:
print(efficientnet_b2_model)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

# Eğitime Başlatma

In [16]:
duraklama noktası

SyntaxError: invalid syntax (3199831575.py, line 1)

### Resnet18 Eğitimi

In [ ]:
resnet18_trained_model = train_model(resnet18_model, dataloaders, resnet18_criterion, resnet18_optimizer, device, num_epochs=300, patience=50, class_names=class_names)

Sınıflar: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Effusion', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumothorax']

Epoch 1/300
--------------------
Train Loss: 2.3335 Acc: 0.1276
Validation Loss: 2.4345 Acc: 0.1540
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 56.00% (28/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 12.00% (6/50)
Sınıf Nodule: 0.00% (0/50)
Sınıf Pleural_Thickening: 76.00% (38/50)
Sınıf Pneumothorax: 10.00% (5/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.18      0.56      0.27        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5      

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.2802 Acc: 0.1475
Validation Loss: 2.2489 Acc: 0.1360
1 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 24.00% (12/50)
Sınıf Cardiomegaly: 34.00% (17/50)
Sınıf Consolidation: 14.00% (7/50)
Sınıf Effusion: 14.00% (7/50)
Sınıf Infiltration: 8.00% (4/50)
Sınıf Mass: 34.00% (17/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 0.00% (0/50)
Sınıf Pleural_Thickening: 2.00% (1/50)
Sınıf Pneumothorax: 6.00% (3/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.18      0.24      0.20        50
           1       0.15      0.34      0.20        50
           2       0.24      0.14      0.18        50
           3       0.14      0.14      0.14        50
           4       0.08      0.08      0.08        50
           5       0.10      0.34      0.15        50
           6       0.00      0.00      0.00        50
           7       0.00      0.00      0.00        50
           8       0.25      0.02      0.04  

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.2548 Acc: 0.1633
Validation Loss: 2.2925 Acc: 0.1740
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 8.00% (4/50)
Sınıf Cardiomegaly: 36.00% (18/50)
Sınıf Consolidation: 80.00% (40/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 28.00% (14/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 22.00% (11/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.33      0.08      0.13        50
           1       0.28      0.36      0.31        50
           2       0.14      0.80      0.24        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        50
           7       0.19      0.28      0.22        50
           8       0.00      0.00      0.00  

...

Train Loss: 2.1810 Acc: 0.2069
Validation Loss: 2.2737 Acc: 0.1400
3 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 2.00% (1/50)
Sınıf Cardiomegaly: 40.00% (20/50)
Sınıf Consolidation: 18.00% (9/50)
Sınıf Effusion: 14.00% (7/50)
Sınıf Infiltration: 10.00% (5/50)
Sınıf Mass: 8.00% (4/50)
Sınıf No Finding: 46.00% (23/50)
Sınıf Nodule: 0.00% (0/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 2.00% (1/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.20      0.02      0.04        50
           1       0.27      0.40      0.33        50
           2       0.10      0.18      0.12        50
           3       0.29      0.14      0.19        50
           4       0.10      0.10      0.10        50
           5       0.12      0.08      0.10        50
           6       0.11      0.46      0.17        50
           7       0.00      0.00      0.00        50
           8       0.00      0.00      0.00   

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.1499 Acc: 0.2209
Validation Loss: 2.2909 Acc: 0.1880
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 22.00% (11/50)
Sınıf Cardiomegaly: 76.00% (38/50)
Sınıf Consolidation: 14.00% (7/50)
Sınıf Effusion: 24.00% (12/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 2.00% (1/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 26.00% (13/50)
Sınıf Pleural_Thickening: 2.00% (1/50)
Sınıf Pneumothorax: 22.00% (11/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.18      0.22      0.20        50
           1       0.16      0.76      0.26        50
           2       0.19      0.14      0.16        50
           3       0.22      0.24      0.23        50
           4       0.00      0.00      0.00        50
           5       0.20      0.02      0.04        50
           6       0.00      0.00      0.00        50
           7       0.37      0.26      0.31        50
           8       0.25      0.02      0.0

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.0942 Acc: 0.2358
Validation Loss: 2.2593 Acc: 0.1740
1 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 44.00% (22/50)
Sınıf Consolidation: 36.00% (18/50)
Sınıf Effusion: 2.00% (1/50)
Sınıf Infiltration: 8.00% (4/50)
Sınıf Mass: 4.00% (2/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 6.00% (3/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 74.00% (37/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.38      0.44      0.41        50
           2       0.22      0.36      0.27        50
           3       0.33      0.02      0.04        50
           4       0.25      0.08      0.12        50
           5       0.33      0.04      0.07        50
           6       0.00      0.00      0.00        50
           7       0.20      0.06      0.09        50
           8       0.00      0.00      0.00    

In [18]:
save_model(resnet18_trained_model, 'resnet18_model.pth')

Model başarıyla kaydedildi: resnet18_model.pth.pth


### CustomCNN Eğitimi

In [33]:
scratch_trained_model = train_model(scratch_model, dataloaders, scratch_criterion, scratch_optimizer, device, num_epochs=20, patience=999, class_names=class_names)

Sınıflar: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Effusion', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumothorax']

Epoch 1/20
--------------------


Train Loss: 3.0414 Acc: 0.0972
Validation Loss: 2.3068 Acc: 0.0980
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 98.00% (49/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 0.00% (0/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 0.00% (0/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 0.00% (0/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.10      0.98      0.18        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        50
           7       0.00      0.00      0.00        50
           8       0.00      0.00      0.00        

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.3033 Acc: 0.1005
Validation Loss: 2.3016 Acc: 0.0980
1 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 0.00% (0/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 98.00% (49/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 0.00% (0/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        50
           7       0.10      0.98      0.18        50
           8       0.00      0.00      0.00        

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

...

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.3028 Acc: 0.0926
Validation Loss: 2.3026 Acc: 0.1000
4 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 0.00% (0/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 100.00% (50/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 0.00% (0/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 0.00% (0/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50
           3       0.00      0.00      0.00        50
           4       0.10      1.00      0.18        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        50
           7       0.00      0.00      0.00        50
           8       0.00      0.00      0.00       

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.3028 Acc: 0.0914
Validation Loss: 2.3026 Acc: 0.1000
5 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 0.00% (0/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 100.00% (50/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 0.00% (0/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 0.00% (0/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50
           3       0.00      0.00      0.00        50
           4       0.10      1.00      0.18        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        50
           7       0.00      0.00      0.00        50
           8       0.00      0.00      0.00       

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

In [34]:
save_model(scratch_trained_model, 'scratch_model.pth')

Model başarıyla kaydedildi: scratch_model.pth.pth


### SimpleCNN Eğitimi

In [35]:
SimpleCNN_trained_model = train_model(SimpleCNN_model, dataloaders, SimpleCNN_criterion, SimpleCNN_optimizer, device, num_epochs=60, patience=999, class_names=class_names)

Sınıflar: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Effusion', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumothorax']

Epoch 1/60
--------------------
Train Loss: 2.1492 Acc: 0.2168
Validation Loss: 2.2285 Acc: 0.1760
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 18.00% (9/50)
Sınıf Cardiomegaly: 24.00% (12/50)
Sınıf Consolidation: 38.00% (19/50)
Sınıf Effusion: 22.00% (11/50)
Sınıf Infiltration: 12.00% (6/50)
Sınıf Mass: 4.00% (2/50)
Sınıf No Finding: 6.00% (3/50)
Sınıf Nodule: 22.00% (11/50)
Sınıf Pleural_Thickening: 6.00% (3/50)
Sınıf Pneumothorax: 24.00% (12/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.10      0.18      0.13        50
           1       0.43      0.24      0.31        50
           2       0.22      0.38      0.28        50
           3       0.13      0.22      0.16        50
           4       0.11      0.12      0.11        50
           5 

In [36]:
save_model(SimpleCNN_trained_model, 'SimpleCNN_model_60')

Model başarıyla kaydedildi: SimpleCNN_model_60.pth


### SimplerCNN2 eğitimi

In [ ]:
SimplerCNN2_trained_model = train_model(SimplerCNN2_model, dataloaders, SimplerCNN2_criterion, SimplerCNN2_optimizer, device, num_epochs=5, patience=999, class_names=class_names)

Sınıflar: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumothorax']

Epoch 1/5
--------------------
Train Loss: 2.2343 Acc: 0.1210
Validation Loss: 2.1837 Acc: 0.1711
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 48.00% (24/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf Nodule: 0.00% (0/50)
Sınıf Pleural_Thickening: 40.00% (20/50)
Sınıf Pneumothorax: 66.00% (33/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.18      0.48      0.26        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.1853 Acc: 0.1331
Validation Loss: 2.1622 Acc: 0.1578
1 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 76.00% (38/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf Nodule: 0.00% (0/50)
Sınıf Pleural_Thickening: 50.00% (25/50)
Sınıf Pneumothorax: 16.00% (8/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.17      0.76      0.27        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        50
           7       0.15      0.50      0.23        50
           8       0.19      0.16      0.17        50

    accuracy            

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.1726 Acc: 0.1441
Validation Loss: 2.1652 Acc: 0.1911
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 16.00% (8/50)
Sınıf Cardiomegaly: 2.00% (1/50)
Sınıf Consolidation: 46.00% (23/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf Nodule: 50.00% (25/50)
Sınıf Pleural_Thickening: 22.00% (11/50)
Sınıf Pneumothorax: 36.00% (18/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.17      0.16      0.17        50
           1       0.09      0.02      0.03        50
           2       0.24      0.46      0.31        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.22      0.50      0.30        50
           7       0.13      0.22      0.17        50
           8       0.18      0.36      0.24        50

    accuracy        

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.1685 Acc: 0.1482
Validation Loss: 2.1558 Acc: 0.1600
1 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 2.00% (1/50)
Sınıf Cardiomegaly: 4.00% (2/50)
Sınıf Consolidation: 52.00% (26/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 4.00% (2/50)
Sınıf Nodule: 26.00% (13/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 56.00% (28/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.03      0.02      0.02        50
           1       0.15      0.04      0.06        50
           2       0.19      0.52      0.28        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5       0.13      0.04      0.06        50
           6       0.24      0.26      0.25        50
           7       0.00      0.00      0.00        50
           8       0.14      0.56      0.22        50

    accuracy           

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.1650 Acc: 0.1501
Validation Loss: 2.1377 Acc: 0.1711
2 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 2.00% (1/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 68.00% (34/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 6.00% (3/50)
Sınıf Mass: 0.00% (0/50)
Sınıf Nodule: 54.00% (27/50)
Sınıf Pleural_Thickening: 8.00% (4/50)
Sınıf Pneumothorax: 16.00% (8/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.14      0.02      0.04        50
           1       0.00      0.00      0.00        50
           2       0.16      0.68      0.26        50
           3       0.00      0.00      0.00        50
           4       0.09      0.06      0.07        50
           5       0.00      0.00      0.00        50
           6       0.20      0.54      0.29        50
           7       0.15      0.08      0.10        50
           8       0.30      0.16      0.21        50

    accuracy            

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

### fine-tuning

In [ ]:
SimplerCNN2_trained_model = train_model(SimplerCNN2_trained_model, dataloaders, SimplerCNN2_criterion, SimplerCNN2_optimizer, device, num_epochs=10, patience=999, class_names=class_names)

Sınıflar: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumothorax']

Epoch 1/10
--------------------
Train Loss: 2.1731 Acc: 0.1443
Validation Loss: 2.1441 Acc: 0.1711
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 10.00% (5/50)
Sınıf Cardiomegaly: 6.00% (3/50)
Sınıf Consolidation: 72.00% (36/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf Nodule: 46.00% (23/50)
Sınıf Pleural_Thickening: 8.00% (4/50)
Sınıf Pneumothorax: 12.00% (6/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.15      0.10      0.12        50
           1       0.33      0.06      0.10        50
           2       0.15      0.72      0.25        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
          

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.1662 Acc: 0.1535
Validation Loss: 2.1226 Acc: 0.1778
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 4.00% (2/50)
Sınıf Cardiomegaly: 4.00% (2/50)
Sınıf Consolidation: 66.00% (33/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 6.00% (3/50)
Sınıf Mass: 0.00% (0/50)
Sınıf Nodule: 70.00% (35/50)
Sınıf Pleural_Thickening: 4.00% (2/50)
Sınıf Pneumothorax: 6.00% (3/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.20      0.04      0.07        50
           1       0.50      0.04      0.07        50
           2       0.18      0.66      0.28        50
           3       0.00      0.00      0.00        50
           4       0.12      0.06      0.08        50
           5       0.00      0.00      0.00        50
           6       0.18      0.70      0.28        50
           7       0.33      0.04      0.07        50
           8       0.14      0.06      0.08        50

    accuracy             

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.1694 Acc: 0.1472
Validation Loss: 2.1315 Acc: 0.1778
1 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 8.00% (4/50)
Sınıf Cardiomegaly: 6.00% (3/50)
Sınıf Consolidation: 66.00% (33/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf Nodule: 70.00% (35/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 10.00% (5/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.19      0.08      0.11        50
           1       0.18      0.06      0.09        50
           2       0.19      0.66      0.29        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.20      0.70      0.31        50
           7       0.00      0.00      0.00        50
           8       0.12      0.10      0.11        50

    accuracy            

...

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.1610 Acc: 0.1578
Validation Loss: 2.1180 Acc: 0.1956
2 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 44.00% (22/50)
Sınıf Consolidation: 56.00% (28/50)
Sınıf Effusion: 12.00% (6/50)
Sınıf Infiltration: 2.00% (1/50)
Sınıf Mass: 0.00% (0/50)
Sınıf Nodule: 62.00% (31/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 0.00% (0/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.20      0.44      0.27        50
           2       0.19      0.56      0.29        50
           3       0.23      0.12      0.16        50
           4       0.10      0.02      0.03        50
           5       0.00      0.00      0.00        50
           6       0.21      0.62      0.32        50
           7       0.00      0.00      0.00        50
           8       0.00      0.00      0.00        50

    accuracy          

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.1561 Acc: 0.1532
Validation Loss: 2.1186 Acc: 0.1889
3 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 2.00% (1/50)
Sınıf Cardiomegaly: 40.00% (20/50)
Sınıf Consolidation: 48.00% (24/50)
Sınıf Effusion: 6.00% (3/50)
Sınıf Infiltration: 4.00% (2/50)
Sınıf Mass: 0.00% (0/50)
Sınıf Nodule: 68.00% (34/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 2.00% (1/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.50      0.02      0.04        50
           1       0.19      0.40      0.25        50
           2       0.20      0.48      0.28        50
           3       0.27      0.06      0.10        50
           4       0.12      0.04      0.06        50
           5       0.00      0.00      0.00        50
           6       0.20      0.68      0.30        50
           7       0.00      0.00      0.00        50
           8       0.05      0.02      0.03        50

    accuracy           

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

### densenet161 eğitimi

In [21]:
densenet161_trained_model = train_model(densenet161_model, dataloaders, densenet161_criterion, densenet161_optimizer, device, num_epochs=50, patience=100, class_names=class_names)

Sınıflar: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Effusion', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumothorax']

Epoch 1/50
--------------------
Train Loss: 2.3439 Acc: 0.1139
Validation Loss: 2.2958 Acc: 0.1280
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 16.00% (8/50)
Sınıf Consolidation: 50.00% (25/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 18.00% (9/50)
Sınıf Mass: 42.00% (21/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 2.00% (1/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 0.00% (0/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.12      0.16      0.14        50
           2       0.15      0.50      0.23        50
           3       0.00      0.00      0.00        50
           4       0.11      0.18      0.14        50
           5       

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.3224 Acc: 0.1266
Validation Loss: 2.3088 Acc: 0.1040
1 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 2.00% (1/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 8.00% (4/50)
Sınıf Effusion: 20.00% (10/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 18.00% (9/50)
Sınıf Pleural_Thickening: 56.00% (28/50)
Sınıf Pneumothorax: 0.00% (0/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.03      0.02      0.03        50
           1       0.00      0.00      0.00        50
           2       0.15      0.08      0.11        50
           3       0.11      0.20      0.14        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        50
           7       0.12      0.18      0.14        50
           8       0.10      0.56      0.17     

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.3123 Acc: 0.1311
Validation Loss: 2.3576 Acc: 0.1280
2 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 8.00% (4/50)
Sınıf Effusion: 36.00% (18/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 74.00% (37/50)
Sınıf Pleural_Thickening: 6.00% (3/50)
Sınıf Pneumothorax: 4.00% (2/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.19      0.08      0.11        50
           3       0.13      0.36      0.19        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        50
           7       0.12      0.74      0.20        50
           8       0.20      0.06      0.09      

...

Train Loss: 2.2865 Acc: 0.1421
Validation Loss: 2.2489 Acc: 0.1680
2 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 24.00% (12/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 8.00% (4/50)
Sınıf Effusion: 42.00% (21/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 46.00% (23/50)
Sınıf Pleural_Thickening: 8.00% (4/50)
Sınıf Pneumothorax: 40.00% (20/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.11      0.24      0.15        50
           1       0.00      0.00      0.00        50
           2       0.44      0.08      0.14        50
           3       0.18      0.42      0.26        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        50
           7       0.17      0.46      0.24        50
           8       0.21      0.08      0.12  

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.2689 Acc: 0.1461
Validation Loss: 2.2422 Acc: 0.1780
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 8.00% (4/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 46.00% (23/50)
Sınıf Effusion: 20.00% (10/50)
Sınıf Infiltration: 8.00% (4/50)
Sınıf Mass: 8.00% (4/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 48.00% (24/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 40.00% (20/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.07      0.08      0.07        50
           1       0.00      0.00      0.00        50
           2       0.22      0.46      0.30        50
           3       0.20      0.20      0.20        50
           4       0.15      0.08      0.10        50
           5       0.16      0.08      0.11        50
           6       0.00      0.00      0.00        50
           7       0.18      0.48      0.26        50
           8       0.00      0.00      0.00  

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

KeyboardInterrupt: 

In [ ]:
save_model(densenet161_trained_model, 'densenet161_model.pth')

In [ ]:
efficientnet_b2_trained_model = train_model(efficientnet_b2_model, dataloaders, efficientnet_b2_criterion, efficientnet_b2_optimizer, device, num_epochs=50, patience=10, class_names=class_names)

In [ ]:
save_model(efficientnet_b2_trained_model, 'efficientnet_b2_model.pth')

## Modeli yniden oluştur

In [ ]:
# model = SimpleCNN(num_classes=len(class_names))  # Model sınıfınızı yeniden tanımlayın
# model.load_state_dict(torch.load("trained_model.pth"))
# model = model.to(device)
# model.eval()  # Modeli değerlendirme moduna alın
# print("Model başarıyla yüklendi.")

## Modeli Kullanma

In [ ]:
# with torch.no_grad():
#     inputs = ...  # Yeni bir veri
#     inputs = inputs.to(device)
#     outputs = model(inputs)
#     _, preds = torch.max(outputs, 1)
#     print(f"Tahmin edilen sınıf: {class_names[preds[0]]}")